In [12]:
# Simulations based on random sampling

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from pandas_datareader import data as web
plt.style.use('ggplot')

In [5]:
 def get_data(ticker, start, end):
   data = web.get_data_yahoo(ticker, start, end)
   data = data['Adj Close']
   returns = data.pct_change()
   meanReturns = returns.mean()
   covMatrix = returns.cov()

   return meanReturns, covMatrix


stockList = ['MMM', 'T', 'WM', 'KO']
# Include a list comprehension for non-american stock exchanges
# tickers = [ticker + '.L' for stock in stockList]

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=300) 

In [ ]:
meanReturns, covMatrix = get_data(stockList, startDate, endDate)

weights = np.random.random(len(meanReturns))          # Random weights
weights /= np.sum(weights)                            # Constrain sum weights = 1


In [17]:
# Set up Monte Carlo environment

simulations = 1000                                    # How many sims
T = 100                                               # How many days
investment = 100000
retrieveArray = np.full(shape=(T, len(weights)), fill_value=meanReturns)
retrieveArray = retrieveArray.T                       # Transpose the array you fetch data from

pf_simulationArray = np.full(shape=(T, simulations), fill_value=0.0)


In [ ]:
# Monte Carlo loop

for m in range(0, simulations):
  zNorm = np.random.normal(size=(T, len(weights)))        # Random normal 
  lower = np.linalg.cholesky(covMatrix)                   # Lower triangle

  # Portfolio multi variate daily return, as a product of the cholesky decomposition
  dailyReturns = retrieveArray + np.inner(lower, zNorm)   

  # Record and accumulate the daily returns to product T length simulations
  pf_simulationArray[:, m] = np.cumprod(np.inner(weights, dailyReturns.T)+1) * investment


plt.figure(figsize=(14,12))
plt.ylabel('Portfolio Value ($)')
plt.xlabel('Date')
plt.title(f'{simulations} Monte Carlo Simulation of Portfolio Value over {T} days')
plt.plot(pf_simulationArray)